In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# from typing import List, Tuple, Optional
# import re
# from dataclasses import dataclass
# import torch
# import torch.nn.functional as F
# from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
# import numpy as np
# import pandas as pd
# from tqdm import tqdm

# @dataclass
# class Product:
#     id: str
#     name: str
#     category_path: List[str]


# class PrefixTreeNode:
#     def __init__(self, token: str):
#         self.token = token
#         self.children = {}
#         self.products = []
    
#     def add_child(self, token: str):
#         if token not in self.children:
#             self.children[token] = PrefixTreeNode(token)
#         return self.children[token]


# class SemanticEncoder:
#     """Encoder để tạo embeddings cho semantic search"""
#     def __init__(self, model_name: str = "Qwen/Qwen3-Embedding-0.6B", device: str = None):
#         print("Loading Semantic Encoder...")
#         if device is None:
#             device = "cuda" if torch.cuda.is_available() else "cpu"
        
#         self.device = device
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model = AutoModel.from_pretrained(model_name).to(device).eval()
#         print(f"✅ Semantic Encoder loaded on {device}")
    
#     def encode(self, texts: List[str]) -> np.ndarray:
#         """Encode texts thành embeddings"""
#         with torch.no_grad():
#             inputs = self.tokenizer(texts, padding=True, truncation=True, 
#                                   return_tensors="pt", max_length=512)
#             inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
#             outputs = self.model(**inputs)
#             embeddings = outputs.last_hidden_state.mean(dim=1)
#             embeddings = F.normalize(embeddings, p=2, dim=1)
            
#         return embeddings.cpu().numpy()


# class Qwen3CrossEncoder:
#     def __init__(self, model_name: str = "Qwen/Qwen3-Reranker-0.6B", device: str = None):
#         print("oading Qwen3-Reranker...")
#         if device is None:
#             device = "cuda" if torch.cuda.is_available() else "cpu"
        
#         self.device = device
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
#         self.model = AutoModelForCausalLM.from_pretrained(model_name).to(device).eval()
        
#         self.token_false_id = self.tokenizer.convert_tokens_to_ids("no")
#         self.token_true_id = self.tokenizer.convert_tokens_to_ids("yes")
#         self.max_length = 8192
        
#         self.prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
#         self.suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
#         self.prefix_tokens = self.tokenizer.encode(self.prefix, add_special_tokens=False)
#         self.suffix_tokens = self.tokenizer.encode(self.suffix, add_special_tokens=False)
        
#         self.task_instruction = "Given a product search query, judge whether the product matches the user's search intent"
#         print(f"✅ Cross-encoder loaded on {device}")
    
#     def format_instruction(self, query: str, doc: str) -> str:
#         return "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
#             instruction=self.task_instruction, query=query, doc=doc
#         )
    
#     def process_inputs(self, pairs: List[str]):
#         inputs = self.tokenizer(
#             pairs, padding=False, truncation='longest_first',
#             return_attention_mask=False, 
#             max_length=self.max_length - len(self.prefix_tokens) - len(self.suffix_tokens)
#         )
        
#         for i, ele in enumerate(inputs['input_ids']):
#             inputs['input_ids'][i] = self.prefix_tokens + ele + self.suffix_tokens
        
#         inputs = self.tokenizer.pad(inputs, padding=True, return_tensors="pt", max_length=self.max_length)
        
#         for key in inputs:
#             inputs[key] = inputs[key].to(self.device)
        
#         return inputs
    
#     def compute_logits(self, inputs):
#         with torch.no_grad():
#             batch_scores = self.model(**inputs).logits[:, -1, :]
#             true_vector = batch_scores[:, self.token_true_id]
#             false_vector = batch_scores[:, self.token_false_id]
#             batch_scores = torch.stack([false_vector, true_vector], dim=1)
#             batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
#             scores = batch_scores[:, 1].exp().tolist()
#         return scores
    
#     def predict(self, pairs: List[List[str]]) -> List[float]:
#         formatted_pairs = [self.format_instruction(query, doc) for query, doc in pairs]
#         inputs = self.process_inputs(formatted_pairs)
#         scores = self.compute_logits(inputs)
#         return scores


# def parse_categories(category_str: str) -> List[str]:
#     """Parse danh mục từ string"""
#     category_str = category_str.strip()
#     if category_str.startswith('[') and category_str.endswith(']'):
#         category_str = category_str[1:-1]
#     categories = [cat.strip() for cat in category_str.split(',')]
#     return categories


# def normalize_text(text: str) -> List[str]:
#     """Chuẩn hóa và tách từ"""
#     text = text.lower()
#     tokens = re.findall(r'\w+', text)
#     return tokens


# def build_prefix_tree(products: List[Product]) -> PrefixTreeNode:
#     """Xây dựng prefix tree"""
#     root = PrefixTreeNode("ROOT")
    
#     for product in products:
#         current = root
#         full_path = " ".join(product.category_path)
#         tokens = normalize_text(full_path)
        
#         # Loại bỏ duplicate liên tiếp
#         unique_tokens = []
#         for token in tokens:
#             if not unique_tokens or token != unique_tokens[-1]:
#                 unique_tokens.append(token)
        
#         for token in unique_tokens:
#             current = current.add_child(token)
        
#         current.products.append(product)
    
#     return root


# def find_matching_node(tree: PrefixTreeNode, query: str) -> Optional[PrefixTreeNode]:
#     """Tìm node sâu nhất match với query"""
#     query_tokens = normalize_text(query)
    
#     current = tree
#     deepest_with_products = None
    
#     for token in query_tokens:
#         if token in current.children:
#             current = current.children[token]
#             if current.products:
#                 deepest_with_products = current
#         else:
#             break
    
#     return deepest_with_products


# def lexical_search(query: str, products: List[Product], top_n: int = 50) -> List[Product]:
#     """Lexical search dựa trên token matching"""
#     query_tokens = normalize_text(query)
    
#     scores = []
#     for product in products:
#         all_text = product.name + " " + " ".join(product.category_path)
#         product_tokens = normalize_text(all_text)
        
#         matches = sum(1 for qt in query_tokens if qt in product_tokens)
#         score = matches / len(query_tokens) if query_tokens else 0
        
#         scores.append((product, score))
    
#     scores.sort(key=lambda x: x[1], reverse=True)
#     return [p for p, s in scores[:top_n] if s > 0]


# def semantic_search(query: str, products: List[Product], 
#                     semantic_encoder: SemanticEncoder, top_n: int = 50) -> List[Product]:
#     """Semantic search dựa trên embeddings"""
#     query_emb = semantic_encoder.encode([query])[0]
    
#     texts = [p.name for p in products]
#     product_embs = semantic_encoder.encode(texts)
    
#     similarities = [float(np.dot(query_emb, emb)) for emb in product_embs]
    
#     scored = list(zip(products, similarities))
#     scored.sort(key=lambda x: x[1], reverse=True)
    
#     return [p for p, s in scored[:top_n]]


# def method_prefix_lexical_cross(query: str, categories: List[str],
#                                  cross_encoder: Qwen3CrossEncoder,
#                                  idx: int) -> Tuple[str, float, str]:
#     """Phương pháp 1: Prefix Tree → Lexical Search → Cross-encoder"""
#     # Tạo products
#     products = [Product(f"{idx}_{i}", cat, [cat]) for i, cat in enumerate(categories)]
    
#     # Build prefix tree
#     tree = build_prefix_tree(products)
    
#     # Bước 1: Prefix matching
#     matched_node = find_matching_node(tree, query)
    
#     if matched_node and matched_node.products:
#         candidates = matched_node.products
#         retrieval_method = "prefix"
#     else:
#         # Bước 2: Lexical search (fallback)
#         candidates = lexical_search(query, products, top_n=50)
#         retrieval_method = "lexical"
    
#     if not candidates:
#         return categories[0], 0.0, "no_candidates"
    
#     # Bước 3: Cross-encoder
#     pairs = [[query, p.name] for p in candidates]
#     scores = cross_encoder.predict(pairs)
    
#     best_idx = np.argmax(scores)
#     return candidates[best_idx].name, scores[best_idx], retrieval_method


# def method_prefix_semantic_cross(query: str, categories: List[str],
#                                   semantic_encoder: SemanticEncoder,
#                                   cross_encoder: Qwen3CrossEncoder,
#                                   idx: int) -> Tuple[str, float, str]:
#     """Phương pháp 2: Prefix Tree → Semantic Search → Cross-encoder (MỚI)"""
#     # Tạo products
#     products = [Product(f"{idx}_{i}", cat, [cat]) for i, cat in enumerate(categories)]
    
#     # Build prefix tree
#     tree = build_prefix_tree(products)
    
#     # Bước 1: Prefix matching
#     matched_node = find_matching_node(tree, query)
    
#     if matched_node and matched_node.products:
#         candidates = matched_node.products
#         retrieval_method = "prefix"
#     else:
#         # Bước 2: Semantic search (fallback) - THAY ĐỔI CHỖ NÀY
#         candidates = semantic_search(query, products, semantic_encoder, top_n=50)
#         retrieval_method = "semantic"
    
#     if not candidates:
#         return categories[0], 0.0, "no_candidates"
    
#     # Bước 3: Cross-encoder
#     pairs = [[query, p.name] for p in candidates]
#     scores = cross_encoder.predict(pairs)
    
#     best_idx = np.argmax(scores)
#     return candidates[best_idx].name, scores[best_idx], retrieval_method


# def compare_all_methods(csv_path: str, device: str = "cpu", verbose: bool = False):
#     """
#     So sánh 2 phương pháp:
#     1. Prefix → Lexical → Cross 
#     2. Prefix → Semantic → Cross 
#     """
#     print("=" * 80)
#     print("COMPARING: LEXICAL vs SEMANTIC SEARCH (in Prefix+Retrieval+Cross pipeline)")
#     print("=" * 80)
    
#     # Load data
#     df = pd.read_csv(csv_path)
#     print(f"\n✅ Loaded {len(df)} test cases")
    
#     # Initialize models
#     print("\n" + "=" * 80)
#     print("INITIALIZING MODELS")
#     print("=" * 80)
    
#     semantic_encoder = SemanticEncoder(device=device)
#     cross_encoder = Qwen3CrossEncoder(device=device)
    
#     # Results storage
#     results = []
    
#     print("\n" + "=" * 80)
#     print("RUNNING TESTS")
#     print("=" * 80)
    
#     # Progress bar
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
#         query = row['tên hàng hóa cần map']
#         expected = row['kết quả mong muốn']
#         categories_str = row['danh mục']
#         categories = parse_categories(categories_str)
        
#         if verbose:
#             print(f"\n--- Test {idx + 1}/{len(df)} ---")
#             print(f"Query: {query}")
#             print(f"Expected: {expected}")
        
#         # METHOD 1: Prefix → Lexical → Cross
#         pred1, score1, retrieval1 = method_prefix_lexical_cross(
#             query, categories, cross_encoder, idx
#         )
#         correct1 = (pred1.strip().lower() == expected.strip().lower())
        
#         # METHOD 2: Prefix → Semantic → Cross
#         pred2, score2, retrieval2 = method_prefix_semantic_cross(
#             query, categories, semantic_encoder, cross_encoder, idx
#         )
#         correct2 = (pred2.strip().lower() == expected.strip().lower())
        
#         if verbose:
#             print(f"  Method 1 (Prefix→Lexical→Cross):  {pred1} [{score1:.3f}] via {retrieval1} {'✅' if correct1 else '❌'}")
#             print(f"  Method 2 (Prefix→Semantic→Cross): {pred2} [{score2:.3f}] via {retrieval2} {'✅' if correct2 else '❌'}")
        
#         results.append({
#             'query': query,
#             'categories': str(categories),
#             'expected': expected,
            
#             # Method 1: Prefix + Lexical + Cross
#             'lexical_pred': pred1,
#             'lexical_score': score1,
#             'lexical_retrieval': retrieval1,
#             'lexical_correct': correct1,
            
#             # Method 2: Prefix + Semantic + Cross
#             'semantic_pred': pred2,
#             'semantic_score': score2,
#             'semantic_retrieval': retrieval2,
#             'semantic_correct': correct2,
#         })
    
#     # Convert to DataFrame
#     results_df = pd.DataFrame(results)
    
#     # Calculate accuracies
#     acc1 = (results_df['lexical_correct'].sum() / len(results_df)) * 100
#     acc2 = (results_df['semantic_correct'].sum() / len(results_df)) * 100
    
#     # Print summary
#     print("\n" + "=" * 80)
#     print("FINAL RESULTS")
#     print("=" * 80)
    
#     print(f"\n{'Method':<50} {'Accuracy':<12} {'Correct/Total'}")
#     print("-" * 80)
#     print(f"{'1. Prefix → Lexical → Cross-encoder':<50} {acc1:>6.2f}%     {results_df['lexical_correct'].sum():>3}/{len(results_df)}")
#     print(f"{'2. Prefix → Semantic → Cross-encoder':<50} {acc2:>6.2f}%     {results_df['semantic_correct'].sum():>3}/{len(results_df)}")
#     print("-" * 80)
    
#     improvement = acc2 - acc1
#     print(f"\n📊 Semantic over Lexical: {improvement:>+6.2f}%")
    
#     # Retrieval method breakdown
#     print("\n" + "=" * 80)
#     print("RETRIEVAL METHOD BREAKDOWN")
#     print("=" * 80)
    
#     print("\nMethod 1 (Lexical):")
#     retrieval_counts1 = results_df['lexical_retrieval'].value_counts()
#     for method, count in retrieval_counts1.items():
#         pct = count / len(results_df) * 100
#         print(f"  {method}: {count} ({pct:.1f}%)")
    
#     print("\nMethod 2 (Semantic):")
#     retrieval_counts2 = results_df['semantic_retrieval'].value_counts()
#     for method, count in retrieval_counts2.items():
#         pct = count / len(results_df) * 100
#         print(f"  {method}: {count} ({pct:.1f}%)")
    
#     # Save results
#     output_path = csv_path.replace('.csv', '_comparison_lexical_vs_semantic.csv')
#     results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
#     print(f"\n✅ Results saved to: {output_path}")
    
#     # Error analysis - Cases where methods differ
#     print("\n" + "=" * 80)
#     print("CASES WHERE METHODS DIFFER")
#     print("=" * 80)
    
#     diff_results = results_df[results_df['lexical_pred'] != results_df['semantic_pred']]
#     print(f"\nFound {len(diff_results)} cases where predictions differ")
    
#     # Cases where semantic is correct but lexical is wrong
#     semantic_wins = results_df[
#         (~results_df['lexical_correct']) & results_df['semantic_correct']
#     ]
#     print(f"\n🎯 Semantic correct, Lexical wrong: {len(semantic_wins)} cases")
    
#     for idx, row in semantic_wins.head(5).iterrows():
#         print(f"\n  [{idx+1}] Query: '{row['query']}'")
#         print(f"      Expected: '{row['expected']}'")
#         print(f"      Lexical:  '{row['lexical_pred']}' [{row['lexical_score']:.3f}] ❌")
#         print(f"      Semantic: '{row['semantic_pred']}' [{row['semantic_score']:.3f}] ✅")
    
#     # Cases where lexical is correct but semantic is wrong
#     lexical_wins = results_df[
#         results_df['lexical_correct'] & (~results_df['semantic_correct'])
#     ]
#     print(f"\n🎯 Lexical correct, Semantic wrong: {len(lexical_wins)} cases")
    
#     for idx, row in lexical_wins.head(5).iterrows():
#         print(f"\n  [{idx+1}] Query: '{row['query']}'")
#         print(f"      Expected: '{row['expected']}'")
#         print(f"      Lexical:  '{row['lexical_pred']}' [{row['lexical_score']:.3f}] ✅")
#         print(f"      Semantic: '{row['semantic_pred']}' [{row['semantic_score']:.3f}] ❌")
    
#     # Agreement analysis
#     print("\n" + "=" * 80)
#     print("AGREEMENT ANALYSIS")
#     print("=" * 80)
    
#     both_correct = results_df[
#         results_df['lexical_correct'] & results_df['semantic_correct']
#     ]
#     print(f"\nBoth methods correct: {len(both_correct)}/{len(results_df)} ({len(both_correct)/len(results_df)*100:.1f}%)")
    
#     both_wrong_same = results_df[
#         (~results_df['lexical_correct']) & 
#         (~results_df['semantic_correct']) &
#         (results_df['lexical_pred'] == results_df['semantic_pred'])
#     ]
#     print(f"Both wrong (same answer): {len(both_wrong_same)}/{len(results_df)} ({len(both_wrong_same)/len(results_df)*100:.1f}%)")
    
#     both_wrong_diff = results_df[
#         (~results_df['lexical_correct']) & 
#         (~results_df['semantic_correct']) &
#         (results_df['lexical_pred'] != results_df['semantic_pred'])
#     ]
#     print(f"Both wrong (different answer): {len(both_wrong_diff)}/{len(results_df)} ({len(both_wrong_diff)/len(results_df)*100:.1f}%)")
    
#     return results_df


# # ==== MAIN ====
# if __name__ == "__main__":
#     csv_path = "/data/small-language-models/duc/Searching_cate/data/cate_long.csv"
    
#     results = compare_all_methods(
#         csv_path=csv_path,
#         device="cuda",  # Hoặc "cuda"
#         verbose=False   # Set True để xem chi tiết từng test case
#     )

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from typing import List, Tuple, Optional, Dict, Set
import re
from dataclasses import dataclass
from enum import Enum
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import numpy as np
import pandas as pd
from tqdm import tqdm


class CategoryLevel(Enum):
    """Phân loại mức độ ưu tiên của category"""
    PRODUCT_TYPE = 1      # Loại sản phẩm cụ thể (quan trọng nhất)
    ATTRIBUTES = 2        # Đặc tính sản phẩm
    BRAND = 3            # Thương hiệu
    GENERAL = 4          # Danh mục chung


@dataclass
class Product:
    id: str
    name: str
    category_path: List[str]
    level: CategoryLevel = CategoryLevel.GENERAL
    priority_score: float = 0.0


class PrefixTreeNode:
    def __init__(self, token: str):
        self.token = token
        self.children = {}
        self.products = []
    
    def add_child(self, token: str):
        if token not in self.children:
            self.children[token] = PrefixTreeNode(token)
        return self.children[token]


class SemanticEncoder:
    """Encoder để tạo embeddings cho semantic search"""
    def __init__(self, model_name: str = "Qwen/Qwen3-Embedding-0.6B", device: str = None):
        print("Loading Semantic Encoder...")
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device).eval()
        print(f"✅ Semantic Encoder loaded on {device}")
    
    def encode(self, texts: List[str]) -> np.ndarray:
        """Encode texts thành embeddings"""
        with torch.no_grad():
            inputs = self.tokenizer(texts, padding=True, truncation=True, 
                                  return_tensors="pt", max_length=512)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings = F.normalize(embeddings, p=2, dim=1)
            
        return embeddings.cpu().numpy()


class Qwen3CrossEncoder:
    def __init__(self, model_name: str = "Qwen/Qwen3-Reranker-0.6B", device: str = None):
        print("Loading Qwen3-Reranker...")
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(device).eval()
        
        self.token_false_id = self.tokenizer.convert_tokens_to_ids("no")
        self.token_true_id = self.tokenizer.convert_tokens_to_ids("yes")
        self.max_length = 8192
        
        self.prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
        self.suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
        self.prefix_tokens = self.tokenizer.encode(self.prefix, add_special_tokens=False)
        self.suffix_tokens = self.tokenizer.encode(self.suffix, add_special_tokens=False)
        
        self.task_instruction = "Given a product search query, judge whether the product matches the user's search intent"
        print(f"✅ Cross-encoder loaded on {device}")
    
    def format_instruction(self, query: str, doc: str) -> str:
        return "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
            instruction=self.task_instruction, query=query, doc=doc
        )
    
    def process_inputs(self, pairs: List[str]):
        inputs = self.tokenizer(
            pairs, padding=False, truncation='longest_first',
            return_attention_mask=False, 
            max_length=self.max_length - len(self.prefix_tokens) - len(self.suffix_tokens)
        )
        
        for i, ele in enumerate(inputs['input_ids']):
            inputs['input_ids'][i] = self.prefix_tokens + ele + self.suffix_tokens
        
        inputs = self.tokenizer.pad(inputs, padding=True, return_tensors="pt", max_length=self.max_length)
        
        for key in inputs:
            inputs[key] = inputs[key].to(self.device)
        
        return inputs
    
    def compute_logits(self, inputs):
        with torch.no_grad():
            batch_scores = self.model(**inputs).logits[:, -1, :]
            true_vector = batch_scores[:, self.token_true_id]
            false_vector = batch_scores[:, self.token_false_id]
            batch_scores = torch.stack([false_vector, true_vector], dim=1)
            batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
            scores = batch_scores[:, 1].exp().tolist()
        return scores
    
    def predict(self, pairs: List[List[str]]) -> List[float]:
        formatted_pairs = [self.format_instruction(query, doc) for query, doc in pairs]
        inputs = self.process_inputs(formatted_pairs)
        scores = self.compute_logits(inputs)
        return scores


def parse_categories(category_str: str) -> List[str]:
    """Parse danh mục từ string"""
    category_str = category_str.strip()
    if category_str.startswith('[') and category_str.endswith(']'):
        category_str = category_str[1:-1]
    categories = [cat.strip() for cat in category_str.split(',')]
    return categories


def normalize_text(text: str) -> List[str]:
    """Chuẩn hóa và tách từ"""
    text = text.lower()
    tokens = re.findall(r'\w+', text)
    return tokens


def classify_category_level(category: str) -> CategoryLevel:
    """
    Phân loại category theo mức độ ưu tiên
    """
    category_lower = category.lower()
    
    # PRODUCT_TYPE: Loại sản phẩm cụ thể (cao nhất)
    product_type_keywords = [
        'sữa rửa mặt', 'gel rửa mặt', 'tạo bọt', 'foaming', 
        'dạng kem', 'dạng gel', 'cleanser', 'toner', 'serum',
        'kem dưỡng', 'kem chống nắng', 'mặt nạ', 'tẩy trang',
        'nước hoa hồng', 'tẩy tế bào chết', 'xịt khoáng',
        'sữa tắm', 'dầu gội', 'dầu xả', 'dưỡng thể',
        'son môi', 'mascara', 'kem nền', 'phấn phủ'
    ]
    
    for keyword in product_type_keywords:
        if keyword in category_lower:
            return CategoryLevel.PRODUCT_TYPE
    
    # ATTRIBUTES: Đặc tính sản phẩm
    attribute_keywords = [
        'cho da dầu', 'cho da khô', 'cho da nhạy cảm', 'cho da mụn',
        'kiềm dầu', 'cấp ẩm', 'làm sạch sâu', 'dưỡng trắng',
        'chống lão hóa', 'giảm mụn', 'se khít lỗ chân lông',
        'không chứa cồn', 'không chứa paraben', 'organic',
        'thiên nhiên', 'chứa bha', 'salicylic acid'
    ]
    
    for keyword in attribute_keywords:
        if keyword in category_lower:
            return CategoryLevel.ATTRIBUTES
    
    # BRAND: Thương hiệu
    brand_keywords = [
        'la roche-posay', 'laroche', 'effaclar', 'vichy', 
        'eucerin', 'cetaphil', 'simple', 'loreal', 'garnier',
        'innisfree', 'some by mi', 'cocoon'
    ]
    
    for keyword in brand_keywords:
        if keyword in category_lower:
            return CategoryLevel.BRAND
    
    # GENERAL: Mặc định
    return CategoryLevel.GENERAL


def calculate_priority_score(product: Product, query_tokens: List[str]) -> float:
    """
    Tính điểm ưu tiên dựa trên:
    1. Category level (quan trọng nhất)
    2. Token matching với query
    """
    # Base score theo level
    level_scores = {
        CategoryLevel.PRODUCT_TYPE: 100.0,
        CategoryLevel.ATTRIBUTES: 50.0,
        CategoryLevel.BRAND: 30.0,
        CategoryLevel.GENERAL: 10.0
    }
    
    base_score = level_scores[product.level]
    
    # Bonus từ token matching
    category_tokens = normalize_text(product.name)
    matches = sum(1 for qt in query_tokens if qt in category_tokens)
    match_bonus = matches * 5
    
    return base_score + match_bonus


def create_prioritized_products(categories: List[str], idx: int) -> List[Product]:
    """Tạo products với phân loại level và priority score"""
    products = []
    
    for i, cat in enumerate(categories):
        level = classify_category_level(cat)
        product = Product(
            id=f"{idx}_{i}",
            name=cat,
            category_path=[cat],
            level=level,
            priority_score=0.0  # Sẽ được tính sau
        )
        products.append(product)
    
    return products


def build_prefix_tree(products: List[Product]) -> PrefixTreeNode:
    """Xây dựng prefix tree"""
    root = PrefixTreeNode("ROOT")
    
    for product in products:
        current = root
        full_path = " ".join(product.category_path)
        tokens = normalize_text(full_path)
        
        # Loại bỏ duplicate liên tiếp
        unique_tokens = []
        for token in tokens:
            if not unique_tokens or token != unique_tokens[-1]:
                unique_tokens.append(token)
        
        for token in unique_tokens:
            current = current.add_child(token)
        
        current.products.append(product)
    
    return root


def find_matching_node(tree: PrefixTreeNode, query: str) -> Optional[PrefixTreeNode]:
    """Tìm node sâu nhất match với query"""
    query_tokens = normalize_text(query)
    
    current = tree
    deepest_with_products = None
    
    for token in query_tokens:
        if token in current.children:
            current = current.children[token]
            if current.products:
                deepest_with_products = current
        else:
            break
    
    return deepest_with_products


def filter_by_priority(products: List[Product], query_tokens: List[str]) -> List[Product]:
    """
    Lọc candidates theo priority:
    1. Ưu tiên PRODUCT_TYPE
    2. Nếu không có, lấy level cao nhất có sẵn
    3. Trong cùng level, sort theo token matching
    """
    if not products:
        return []
    
    # Tính priority score cho tất cả
    for p in products:
        p.priority_score = calculate_priority_score(p, query_tokens)
    
    # Nhóm theo level
    by_level: Dict[CategoryLevel, List[Product]] = {}
    for p in products:
        if p.level not in by_level:
            by_level[p.level] = []
        by_level[p.level].append(p)
    
    # Ưu tiên PRODUCT_TYPE
    if CategoryLevel.PRODUCT_TYPE in by_level:
        candidates = by_level[CategoryLevel.PRODUCT_TYPE]
    # Nếu không có PRODUCT_TYPE, lấy level cao nhất
    else:
        best_level = min(by_level.keys(), key=lambda x: x.value)
        candidates = by_level[best_level]
    
    # Sort theo priority score
    candidates.sort(key=lambda x: x.priority_score, reverse=True)
    
    return candidates


def lexical_search(query: str, products: List[Product], top_n: int = 50) -> List[Product]:
    """Lexical search với priority filtering"""
    query_tokens = normalize_text(query)
    
    scores = []
    for product in products:
        all_text = product.name + " " + " ".join(product.category_path)
        product_tokens = normalize_text(all_text)
        
        matches = sum(1 for qt in query_tokens if qt in product_tokens)
        score = matches / len(query_tokens) if query_tokens else 0
        
        if score > 0:
            scores.append((product, score))
    
    # Sort theo score
    scores.sort(key=lambda x: x[1], reverse=True)
    candidates = [p for p, s in scores[:top_n]]
    
    # Apply priority filtering
    return filter_by_priority(candidates, query_tokens)[:top_n]


def semantic_search(query: str, products: List[Product], 
                    semantic_encoder: SemanticEncoder, top_n: int = 50) -> List[Product]:
    """Semantic search với priority filtering"""
    query_tokens = normalize_text(query)
    query_emb = semantic_encoder.encode([query])[0]
    
    texts = [p.name for p in products]
    product_embs = semantic_encoder.encode(texts)
    
    similarities = [float(np.dot(query_emb, emb)) for emb in product_embs]
    
    scored = list(zip(products, similarities))
    scored.sort(key=lambda x: x[1], reverse=True)
    
    candidates = [p for p, s in scored[:top_n]]
    
    # Apply priority filtering
    return filter_by_priority(candidates, query_tokens)[:top_n]


def method_prefix_lexical_cross_v2(query: str, categories: List[str],
                                    cross_encoder: Qwen3CrossEncoder,
                                    idx: int) -> Tuple[str, float, str, str]:
    """
    Phương pháp cải tiến với Priority System:
    Prefix Tree → Priority Filtering → Lexical Search → Cross-encoder
    """
    query_tokens = normalize_text(query)
    
    # Tạo products với level classification
    products = create_prioritized_products(categories, idx)
    
    # Build prefix tree
    tree = build_prefix_tree(products)
    
    # Bước 1: Prefix matching
    matched_node = find_matching_node(tree, query)
    
    if matched_node and matched_node.products:
        # Bước 2: Priority filtering
        candidates = filter_by_priority(matched_node.products, query_tokens)
        retrieval_method = "prefix+priority"
        level_info = candidates[0].level.name if candidates else "NONE"
    else:
        # Bước 3: Lexical search (fallback) với priority
        candidates = lexical_search(query, products, top_n=50)
        retrieval_method = "lexical+priority"
        level_info = candidates[0].level.name if candidates else "NONE"
    
    if not candidates:
        return categories[0], 0.0, "no_candidates", "NONE"
    
    # Bước 4: Cross-encoder
    pairs = [[query, p.name] for p in candidates]
    scores = cross_encoder.predict(pairs)
    
    best_idx = np.argmax(scores)
    return candidates[best_idx].name, scores[best_idx], retrieval_method, level_info


def method_prefix_semantic_cross_v2(query: str, categories: List[str],
                                     semantic_encoder: SemanticEncoder,
                                     cross_encoder: Qwen3CrossEncoder,
                                     idx: int) -> Tuple[str, float, str, str]:
    """
    Phương pháp cải tiến với Priority System:
    Prefix Tree → Priority Filtering → Semantic Search → Cross-encoder
    """
    query_tokens = normalize_text(query)
    
    # Tạo products với level classification
    products = create_prioritized_products(categories, idx)
    
    # Build prefix tree
    tree = build_prefix_tree(products)
    
    # Bước 1: Prefix matching
    matched_node = find_matching_node(tree, query)
    
    if matched_node and matched_node.products:
        # Bước 2: Priority filtering
        candidates = filter_by_priority(matched_node.products, query_tokens)
        retrieval_method = "prefix+priority"
        level_info = candidates[0].level.name if candidates else "NONE"
    else:
        # Bước 3: Semantic search (fallback) với priority
        candidates = semantic_search(query, products, semantic_encoder, top_n=50)
        retrieval_method = "semantic+priority"
        level_info = candidates[0].level.name if candidates else "NONE"
    
    if not candidates:
        return categories[0], 0.0, "no_candidates", "NONE"
    
    # Bước 4: Cross-encoder
    pairs = [[query, p.name] for p in candidates]
    scores = cross_encoder.predict(pairs)
    
    best_idx = np.argmax(scores)
    return candidates[best_idx].name, scores[best_idx], retrieval_method, level_info


def compare_all_methods_v2(csv_path: str, device: str = "cpu", verbose: bool = False):
    """
    So sánh phương pháp cũ vs mới (với Priority System):
    1. Old: Prefix → Lexical → Cross
    2. New: Prefix → Priority → Lexical → Cross
    3. New: Prefix → Priority → Semantic → Cross
    """
    print("=" * 80)
    print("COMPARING: Old vs New (WITH PRIORITY SYSTEM)")
    print("=" * 80)
    
    # Load data
    df = pd.read_csv(csv_path)
    print(f"\n✅ Loaded {len(df)} test cases")
    
    # Initialize models
    print("\n" + "=" * 80)
    print("INITIALIZING MODELS")
    print("=" * 80)
    
    semantic_encoder = SemanticEncoder(device=device)
    cross_encoder = Qwen3CrossEncoder(device=device)
    
    # Results storage
    results = []
    
    print("\n" + "=" * 80)
    print("RUNNING TESTS")
    print("=" * 80)
    
    # Progress bar
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        query = row['tên hàng hóa cần map']
        expected = row['kết quả mong muốn']
        categories_str = row['danh mục']
        categories = parse_categories(categories_str)
        
        if verbose:
            print(f"\n--- Test {idx + 1}/{len(df)} ---")
            print(f"Query: {query}")
            print(f"Expected: {expected}")
        
        # NEW METHOD 1: Priority + Lexical
        pred1, score1, retrieval1, level1 = method_prefix_lexical_cross_v2(
            query, categories, cross_encoder, idx
        )
        correct1 = (pred1.strip().lower() == expected.strip().lower())
        
        # NEW METHOD 2: Priority + Semantic
        pred2, score2, retrieval2, level2 = method_prefix_semantic_cross_v2(
            query, categories, semantic_encoder, cross_encoder, idx
        )
        correct2 = (pred2.strip().lower() == expected.strip().lower())
        
        if verbose:
            print(f"  Priority+Lexical:  {pred1} [{score1:.3f}] via {retrieval1} [{level1}] {'✅' if correct1 else '❌'}")
            print(f"  Priority+Semantic: {pred2} [{score2:.3f}] via {retrieval2} [{level2}] {'✅' if correct2 else '❌'}")
        
        results.append({
            'query': query,
            'categories': str(categories),
            'expected': expected,
            
            # Priority + Lexical
            'priority_lexical_pred': pred1,
            'priority_lexical_score': score1,
            'priority_lexical_retrieval': retrieval1,
            'priority_lexical_level': level1,
            'priority_lexical_correct': correct1,
            
            # Priority + Semantic
            'priority_semantic_pred': pred2,
            'priority_semantic_score': score2,
            'priority_semantic_retrieval': retrieval2,
            'priority_semantic_level': level2,
            'priority_semantic_correct': correct2,
        })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Calculate accuracies
    acc1 = (results_df['priority_lexical_correct'].sum() / len(results_df)) * 100
    acc2 = (results_df['priority_semantic_correct'].sum() / len(results_df)) * 100
    
    # Print summary
    print("\n" + "=" * 80)
    print("FINAL RESULTS (WITH PRIORITY SYSTEM)")
    print("=" * 80)
    
    print(f"\n{'Method':<60} {'Accuracy':<12} {'Correct/Total'}")
    print("-" * 80)
    print(f"{'1. Prefix → Priority → Lexical → Cross':<60} {acc1:>6.2f}%     {results_df['priority_lexical_correct'].sum():>3}/{len(results_df)}")
    print(f"{'2. Prefix → Priority → Semantic → Cross':<60} {acc2:>6.2f}%     {results_df['priority_semantic_correct'].sum():>3}/{len(results_df)}")
    print("-" * 80)
    
    improvement = acc2 - acc1
    print(f"\n📊 Semantic over Lexical (with Priority): {improvement:>+6.2f}%")
    
    # Level distribution
    print("\n" + "=" * 80)
    print("CATEGORY LEVEL DISTRIBUTION")
    print("=" * 80)
    
    print("\nMethod 1 (Priority + Lexical):")
    level_counts1 = results_df['priority_lexical_level'].value_counts()
    for level, count in level_counts1.items():
        pct = count / len(results_df) * 100
        correct_at_level = results_df[
            (results_df['priority_lexical_level'] == level) & 
            results_df['priority_lexical_correct']
        ].shape[0]
        acc_at_level = correct_at_level / count * 100 if count > 0 else 0
        print(f"  {level}: {count} ({pct:.1f}%) - Acc: {acc_at_level:.1f}%")
    
    print("\nMethod 2 (Priority + Semantic):")
    level_counts2 = results_df['priority_semantic_level'].value_counts()
    for level, count in level_counts2.items():
        pct = count / len(results_df) * 100
        correct_at_level = results_df[
            (results_df['priority_semantic_level'] == level) & 
            results_df['priority_semantic_correct']
        ].shape[0]
        acc_at_level = correct_at_level / count * 100 if count > 0 else 0
        print(f"  {level}: {count} ({pct:.1f}%) - Acc: {acc_at_level:.1f}%")
    
    # Save results
    output_path = csv_path.replace('.csv', '_priority_system_comparison.csv')
    results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ Results saved to: {output_path}")
    
    # Error analysis
    print("\n" + "=" * 80)
    print("KEY IMPROVEMENTS FROM PRIORITY SYSTEM")
    print("=" * 80)
    
    # Cases improved by priority system
    improved_cases = results_df[
        results_df['priority_lexical_correct'] | results_df['priority_semantic_correct']
    ]
    
    product_type_wins = improved_cases[
        (improved_cases['priority_lexical_level'] == 'PRODUCT_TYPE') |
        (improved_cases['priority_semantic_level'] == 'PRODUCT_TYPE')
    ]
    
    print(f"\n✅ Cases where PRODUCT_TYPE was correctly prioritized: {len(product_type_wins)}")
    
    for idx, row in product_type_wins.head(3).iterrows():
        print(f"\n  [{idx+1}] Query: '{row['query']}'")
        print(f"      Expected: '{row['expected']}'")
        print(f"      Priority+Lexical:  '{row['priority_lexical_pred']}' [{row['priority_lexical_level']}] {'✅' if row['priority_lexical_correct'] else '❌'}")
        print(f"      Priority+Semantic: '{row['priority_semantic_pred']}' [{row['priority_semantic_level']}] {'✅' if row['priority_semantic_correct'] else '❌'}")
    
    return results_df


if __name__ == "__main__":
    # Test với file CSV
    csv_path = "test_cases.csv"  # Thay bằng path thực tế
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    results = compare_all_methods_v2(
        csv_path=csv_path,
        device=device,
        verbose=True  # Set False để chạy nhanh hơn
    )